In [1]:
import networkx as nx
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re

from API import K
import requests
import json
import googlemaps
from dms2dec.dms_convert import dms2dec
import geopy.distance

## Import network and IVS data

In [2]:
# import network
G = pickle.load(open('data/network_digital_twin_v0.3.pickle', 'rb'))

# import cleaned and restructured IVS data
df_ivs = pickle.load( open("data/df_trips_per_path_hourly.p", "rb" ) )

df_h = pd.read_csv('data/cleaned_harbours.csv')
# extract position for drawing purposes
pos_dict = {}
for node in G.nodes:
    pos_dict[node] = (G.nodes[node]['X'],G.nodes[node]['Y'])

#extract data
df_links = nx.to_pandas_edgelist(G)
df_nodes = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

# add degree to dataframes
df_links['degree_source'] = df_links.source.apply(lambda x: G.degree[x])
df_links['degree_target'] = df_links.source.apply(lambda x: G.degree[x])
df_nodes['degree'] = G.degree
df_nodes['degree'] = df_nodes.degree.apply(lambda x: x[1])

In [ ]:
df_ivs.replace(0,np.nan, inplace=True)
# for now only consider 100 most frequented origin destination pairs, may reconsider later
df_ivs = df_ivs.head(100)
# check dataframe
df_ivs

## Step 1: Select relevant harbours and determine decimal degrees coordinates for all habours

In [ ]:
# get unique harbours from IVS dataframe
h_list = list(set(list(df_ivs.origin.unique())) | set(list(df_ivs.destination.unique())))

# get unique city codes (e.g. remove NL prefix)
h_list = [re.sub("NL", "", i) for i in h_list]

#subset harbour data for these harbours and reset index
df_h = df_h.loc[(df_h.city_abbr.isin(h_list)) & (df_h.country == 'NL')]
df_h.reset_index(inplace=True, drop=True)

In [ ]:
len(h_list)

In [ ]:
# fix dtypes
for i in df_h.columns:
    if df_h.dtypes[i] == 'O':
        df_h[i] = df_h[i].astype('|S80')
        df_h[i] = df_h[i].apply(lambda x: x.decode('utf-8'))
# the coordinates are inaccurate and quite a lot of data is missing. This must be fixed first before we continue.

In [ ]:
# #convert latitude and longitude to decimal degrees for harbours with this entry
df_h['lat'] = 0
df_h['lon'] = 0
for key, coords in enumerate(df_h.coords):
    lat_lon=[]
    if coords!='nan':
        for j in range(2):
            if j == 0:
                part_a = coords.split()[j][:2]
                part_b = coords.split()[j][-3:-1]
                cor = (str(part_a)+"°"+str(part_b)+''''0"N"''')
                cor = dms2dec(cor)
                df_h.lat[key] = cor
            else:
                part_a = coords.split()[j][:3]
                part_b = coords.split()[j][-3:-1]
                cor = (str(part_a)+"°"+str(part_b)+''''0"E"''')
                cor = dms2dec(cor)
                df_h.lon[key] = cor

In [ ]:
df_h

In [ ]:
# still missing quite some, try and fetch these using google maps api

### Try to retrieve missing data using google maps

In [ ]:
for i, city in enumerate(df_h.city_full):
    if df_h.coords[i] == 'nan':
        r = requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?address={'Haven', city, 'Nederland'}&key={K}")
        results = json.loads(r.content)
        if 'results' in results.keys():
            if len(results['results'])>0:
                lat_r = results['results'][0]['geometry']['location']['lat']
                lon_r = results['results'][0]['geometry']['location']['lng']
                df_h['lat'][i] = lat_r
                df_h['lon'][i] = lon_r
        else:
            print('No location found for harbour', city)


In [ ]:
df_h.head(100)

### Manually check and fill in last missing data

In [ ]:
# some last manual changes
# Stein mistake google API fetch, finds somewhere near soest somehow
# Wageningen: location fetched at other side of the city
# Geertruidenberg: inland harbour instead of harbour along waal selected
# Genemuiden: inland harbour selected, might be problem because of curve in river around city
# Terneuzen: inland harbour selected somehow
# Farsum and Delfzijl: Delfzijl, mistake in coords, Farsum very closeby, safer to hard code
d_cor_h = {'Stein':[50.974662, 5.756552], 'Wageningen':[51.955027, 5.648670], 'Geertruidenberg': [51.712726, 4.845269], 'Genemuiden':[52.629176, 6.053162], 'Terneuzen':[51.342704, 3.814359], 'Farsum':[53.314251, 6.930846], 'Delfzijl':[53.330089, 6.934031]}

In [ ]:
# fill in manually found values
for harbour in d_cor_h.keys():
    df_h.lat[df_h.loc[df_h.city_full == harbour].index] = d_cor_h[harbour][0]
    df_h.lon[df_h.loc[df_h.city_full == harbour].index] = d_cor_h[harbour][1]

### Basic cleaning: apply bounding box en removing links from i to i

In [ ]:
for node1, node2 in G.edges:
    if node1 == node2:
        print("Self loop identified node", node1)
        G.remove_edge(node1,node2)

In [ ]:
# bounding box nl
bb = (3.31497114423, 50.803721015, 7.09205325687, 53.5104033474)

df_nodes = df_nodes.loc[(df_nodes.X.between(bb[0], bb[2])) & (df_nodes.Y.between(bb[1], bb[3]))]

#visualise new subset

#subset graph and make editable again
G = G.subgraph(df_nodes.index)
G = nx.Graph(G)

### Visually check network and harbours

In [ ]:
#visual check
fig, ax = plt.subplots(dpi=200)
nx.draw_networkx_edges(G, pos_dict)
plt.scatter(df_h.lon,df_h.lat,c='r')
a = df_h.loc[df_h.lon==df_h.lon.min()]
# plt.scatter(a.lon,a.lat,c='b')
berth_nodes = df_nodes.loc[df_nodes.n.str.contains('Berth')].index
berth_nodes = df_nodes.loc[df_nodes.index.isin(berth_nodes)]
plt.scatter(berth_nodes.X,berth_nodes.Y, s=10, c='b')

## Step 2: throw out all small nodes and only keep the largest component

In [ ]:
#extract data
df_links = nx.to_pandas_edgelist(G)
df_nodes = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

In [ ]:
#check out codes to base selection on
df_links.Code.unique()
nodes_to_keep = list(df_links.loc[df_links.Code != '_0'].source) + list(df_links.loc[df_links.Code != '_0'].target)
G = G.subgraph(nodes_to_keep)
G = nx.Graph(G)

In [ ]:
plt.subplots(dpi=200)
nx.draw_networkx_edges(G, pos_dict)

In [ ]:
df_links = nx.to_pandas_edgelist(G)
df_nodes = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

## Step 3: Determine harbour nodes
A harbour node, is a node on the ongoing route along which the harbour is located. This node may be found because of its degree which is higher than 3 and because the link of which it is the source node, has the tag vaarwegvak 0 tot H-0.

In [ ]:
#Check initial range: 5k seems to be okay

In [ ]:
dev = 0.04
a = 51.985103 + dev
b = 5.898730 + dev

In [ ]:
geopy.distance.geodesic((51.985103,5.898730),(a,b))

In [ ]:
# create an additional column to put corresponding harbour node in
df_h['harbour_node'] = 0
# loop over all harbour entries
for i in df_h.index:
    x = df_h.lon[i]
    y = df_h.lat[i]
    dev = 0.04

    #find nodes within deviation
    #select nodes near
    selection = list(df_nodes.loc[(df_nodes.X.between(x-dev, x+dev)) & (df_nodes.Y.between(y-dev, y+dev)) & (df_links.GeoType == 'section')&(df_links.source.str.isdigit())].index)

    # in some areas there are very few nodes, therefore iteratively increase range to look for nodes until at least one is found
    while len(selection) == 0:
        dev+=0.1
        selection = list(df_nodes.loc[(df_nodes.X.between(x-dev, x+dev)) & (df_nodes.Y.between(y-dev, y+dev))].index)

    #select corresponding links and subset links that go to harbour, subset sections with origin is number (no object)
    selection = df_links.loc[((df_links.source.isin(selection))|(df_links.target.isin(selection)))&(df_links.Name == 'Vaarwegvak van 0 tot 0 - H')&(df_links.GeoType == 'section')&(df_links.source.str.isdigit())]

    if len(selection) != 0:
        selection['dist'] = 0
        # if there are items, take nearest, first we need to determine dists
        for j in selection.index:
            sel_source = selection.source[j]
            sel_source = df_nodes.loc[sel_source]
            selection.dist[j]=geopy.distance.geodesic((x,y),(sel_source.X,sel_source.Y))
        # sort by dist and pick firstG
        selection = selection.loc[selection.dist == selection.dist.min()].source
        df_h['harbour_node'][i]= selection.values[0]
    else:
        selection = list(df_nodes.loc[(df_nodes.X.between(x-dev, x+dev)) & (df_nodes.Y.between(y-dev, y+dev))].index)
        if len(selection) != 0:
            selection = df_links.loc[((df_links.source.isin(selection))|(df_links.target.isin(selection)))&(df_links.GeoType == 'section')&(df_links.source.str.isdigit())]
            selection['dist'] = 0
            for j in selection.index:
                sel_source = selection.source[j]
                sel_source = df_nodes.loc[sel_source]
                selection.dist[j]=geopy.distance.geodesic((x,y),(sel_source.X,sel_source.Y))
            # sort by dist and pick firstG
            selection = selection.loc[selection.dist == selection.dist.min()].source
            df_h['harbour_node'][i]= selection.values[0]
        else:
            print('Check entry', df_h.city_full[i])

In [ ]:
harbour_nodes = list(df_h.harbour_node.unique())

In [ ]:
harbour_nodes

In [ ]:
df_h

In [ ]:
plt.subplots(dpi=200)
nx.draw_networkx_edges(G, pos_dict)
nx.draw_networkx_nodes(G, pos_dict, harbour_nodes,node_size=10, node_color='red')

## Step 4: Split flows where necessary
Large ships may not be able to take the same route as small ships, hence these should be observed as separate flows

In [ ]:
types = list(df_links.Code.unique())
types.remove('_0')
types = sorted(types[1:])
print(types)
#sorted in wrong manner, correct manually
types = ['I', 'II', 'III', 'IV', 'V_A', 'V_B', 'VI_A', 'VI_B', 'VI_C']
type_dict={}
#minus 1 because last one is open class anyway
for i in range(len(types)-1):
    type_dict[i] = types[:(i+1)]
type_dict[8]=types

In [ ]:
df_ivs

In [ ]:
#To what extent to paths differ for these OD pairs for different boat types?
routes_dests = {}
routes_types_paths = {}
routes_types_path_lengths = {}
for i in df_ivs.index:

    org_n = df_ivs.origin[i]
    org = df_h.loc[df_h.harbour_code == org_n]['harbour_node'].values[0]
    dest_n = df_ivs.destination[i]
    dest = df_h.loc[df_h.harbour_code == dest_n]['harbour_node'].values[0]

    org_route = nx.dijkstra_path(G, org, dest, weight='length_m')
    routes_types_paths[(org_n, dest_n, 0)] = org_route
    routes_types_path_lengths[(org_n, dest_n, 0)] = nx.dijkstra_path_length(G, org, dest, weight='length_m')
    route_v = {0:types}
    r=0
    for type_index, types_exc in type_dict.items():

        #copy graph
        H = G.copy()

        #determine links that are not available based on dict (e.g. of the type in type_exc list)
        unavailable_edges = df_links.loc[df_links.Code.isin(types_exc)]
        #now remove those edges
        for link in unavailable_edges.index:
            H.remove_edge(unavailable_edges.source[link], unavailable_edges.target[link])

        if nx.has_path(H, org, dest):
            route_type = nx.dijkstra_path(H, org, dest, weight='length_m')
            if route_type != org_route:
                #new route found, r higher now
                r+=1
                #if first time alt: old route is for exc
                if r == 1:
                    route_v[r-1] = tuple(types_exc)
                #if not first time alt: previous route is for all as before minus ones that take new route
                else:
                    route_v[r-1] = tuple(set(route_v[r-1])-(set(types)-set(types_exc)))
                #new route is for all not excluded types
                route_v[r] = tuple(set(types)-set(types_exc))
                #store new route
                routes_types_paths[(org_n, dest_n, r)] = route_type
                routes_types_path_lengths[(org_n, dest_n, r)] = nx.dijkstra_path_length(H, org, dest, weight='length_m')

                #update most recent route
                org_route = route_type

                # print("Ships of type", set(types)-set(types_exc), "must take other route for path", org_n, dest_n)
    #add all routes to main dict to check
    routes_dests[(org_n,dest_n)] = route_v


In [ ]:
routes_dests

In [ ]:
routes_types_paths

In [ ]:
routes_dests = pickle.load(open('data/revised_cleaning_results/users_ship_specific_routes.p', 'rb'))

In [ ]:
len(routes_types_paths.keys())

In [ ]:
#create dict with cumulative rows that are related to each ship type
df_ships = pd.read_excel('data/ship_types.xlsx')

In [ ]:
#pregenerate ship sets for combinations on routes
all_type_comb = []
for i in routes_dests.keys():
    for key, item in routes_dests[i].items():
        if not item in all_type_comb:
            all_type_comb.append(item)

In [ ]:
cor_RWS_types = {}
for type_combi in all_type_comb:
    a = df_ships.loc[df_ships['CEMT-class'].isin(type_combi)]
    a = list(a['RWS-class'].values)
    cor_RWS_types[tuple(type_combi)] = a

In [ ]:
cor_RWS_types
# looks good, now all that's left is to create the exploded df

In [ ]:
#worked, now we must fill a new dict with a row for each route version in this dictionary
# a key for each row of the df
dict_ivs_edited = {i:[] for i in df_ivs.columns}
dict_ivs_edited['route_v'] = []

for org, des in routes_dests.keys():
    #first loc row to process
    for key, types in routes_dests[(org,des)].items():
        for hour in range(24):
            if len(routes_dests[(org,des)].keys()) == 1:
                dict_ivs_edited['route_v'].append(0)
                #if there is only one route, just values to new df
                for column in df_ivs.columns:
                    to_a = df_ivs.loc[(df_ivs.origin == org)&(df_ivs.destination == des)&(df_ivs.hour==hour)].reset_index(drop=True).loc[0, column]
                    dict_ivs_edited[column].append(to_a)
            else:
                #append route version
                dict_ivs_edited['route_v'].append(key)
                #copy total count, org and dest and hour
                to_a = df_ivs.loc[(df_ivs.origin == org)&(df_ivs.destination == des)&(df_ivs.hour==hour)].reset_index(drop=True)
                for column in to_a.iloc[:,:4].columns:
                    dict_ivs_edited[column].append(to_a.loc[0,column])

                # finally copy value if in dict and otherwise set value to 0 for other columns
                # print(types)
                columns_to_copy = cor_RWS_types[tuple(types)]
                all_types = list(to_a.iloc[:,4:].columns)
                for type1 in all_types:
                    if type1 in columns_to_copy:
                        dict_ivs_edited[type1].append(to_a.loc[0,type1])
                    else:
                        dict_ivs_edited[type1].append(0)

In [ ]:
df_ivs_exploded = pd.DataFrame.from_dict(dict_ivs_edited)

In [ ]:
df_ivs_exploded['trip_count']=df_ivs_exploded.iloc[:, 4:-1].sum(axis=1)
df_ivs_exploded

In [ ]:
routes_types_paths = pickle.load(open("data/revised_cleaning_results/paths_ship_specific_routes.p", "rb"))
routes_types_path_lengths = pickle.load(open("data/revised_cleaning_results/path_lengths_ship_specific_routes.p", "rb"))

## Step 5: only keep dijkstra paths between nodes
All paths necessary paths are already generated, all that's left is to find all original nodes

In [ ]:
harbour_nodes = list(df_h.harbour_node.unique())
node_list = []
for key, route in routes_types_paths.items():
    node_list.append(route)

expanded_node_list = [x for xs in node_list for x in xs]
node_list = list(set(expanded_node_list))
G = G.subgraph(node_list)
G = nx.Graph(G)

plt.subplots(dpi=200)
nx.draw_networkx_edges(G, pos_dict)
nx.draw_networkx_nodes(G, pos_dict, harbour_nodes, node_size=10, node_color='red')

In [ ]:
### Remove 0 entries from all datasets
for i in df_ivs_exploded.index:
    if df_ivs_exploded.trip_count[i] == 0:
        try:
            del routes_types_paths[(df_ivs_exploded.origin[i],df_ivs_exploded.destination[i],df_ivs_exploded.route_v[i])]
            del routes_types_path_lengths[(df_ivs_exploded.origin[i],df_ivs_exploded.destination[i],df_ivs_exploded.route_v[i])]
        except:
            print("already gone")
        # try:
        #     df_ivs_exploded=df_ivs_exploded.drop(labels=[i], axis=0)
        # except:
        #     print("already gone")
        del routes_dests[(df_ivs_exploded.origin[i],df_ivs_exploded.destination[i],df_ivs_exploded.route_v[i])]

In [ ]:
df_ivs_exploded.reset_index(inplace=True, drop=True)

In [ ]:
routes_dests

In [ ]:
df_ivs_exploded.loc[df_ivs_exploded.hour==0]

# Code below was used to conclude that different routes had to be considered for OD pairs

### Q: how many links for large ships (e.g. not I, II or III) are not in the graph now?
Also plot links for small ships only in other color to visualise result

In [49]:
# # select nodes from org graph with not _0, I, II or III
# K = pickle.load(open('data/network_digital_twin_v0.3.pickle', 'rb'))
# df_links_or = nx.to_pandas_edgelist(K)
# df_nodes_or = pd.DataFrame.from_dict(dict(K.nodes(data=True)), orient='index')
#
# # bounding box nl
# bb = (3.31497114423, 50.803721015, 7.09205325687, 53.5104033474)
# df_nodes_or = df_nodes_or.loc[(df_nodes_or.X.between(bb[0], bb[2])) & (df_nodes_or.Y.between(bb[1], bb[3]))]
#
# #subset graph and make editable again
# K = K.subgraph(df_nodes_or.index)
# K = nx.Graph(K)
#
# df_links_or = nx.to_pandas_edgelist(K)
# df_nodes_or = pd.DataFrame.from_dict(dict(K.nodes(data=True)), orient='index')
#
# nx.draw_networkx_edges(K, pos_dict)

In [ ]:
# a = df_links_or.loc[~df_links_or.Code.isin(['_0','I','II','III'])]

In [ ]:
# large_edges_full = []
# for i in a.index:
#     large_edges_full.append(tuple([a.source[i], a.target[i]]))

In [ ]:
# len(large_edges_full)

In [ ]:
# large_edges = []
# large_links = df_links.loc[~df_links.Code.isin(['_0','I','II','III'])]
# for i in large_links.index:
#     large_edges.append(tuple([large_links.source[i], large_links.target[i]]))
#
# small_edges = []
# small_links = df_links.loc[df_links.Code.isin(['_0','I','II','III'])]
# for i in small_links.index:
#     small_edges.append(tuple([small_links.source[i], small_links.target[i]]))
#
# for node1, node2 in K.edges:
#     if node1 == node2:
#         print("Self loop identified node", node1)
#         K.remove_edge(node1,node2)
#
# df_links_or = nx.to_pandas_edgelist(K)
# df_nodes_or = pd.DataFrame.from_dict(dict(K.nodes(data=True)), orient='index')
# large_not_in_G = list(set(large_edges_full)-set(large_edges))

In [ ]:
# len(small_edges)

In [ ]:
# len(large_edges)
# # good, counts up till

In [ ]:
# fig, ax = plt.subplots(dpi=200)
# K = K.subgraph(large_not_in_G)
# K = nx.Graph(K)
#
# nx.draw_networkx_edges(G, pos_dict,large_edges, edge_color='g',ax=ax, label='large edges subset')
# nx.draw_networkx_edges(G, pos_dict, small_edges, edge_color='b',ax=ax, label='small edges subset')
# nx.draw_networkx_edges(G, pos_dict, large_not_in_G, edge_color='r',ax=ax, label='large not in subset')
# nx.draw_networkx_nodes(G, pos_dict, harbour_nodes, node_size=10, node_color='y', label='harbours')
# plt.legend(fontsize=8)
# plt.show()
#

## Intermediate checks
1. Are all paths available of df_ivs?
2. Which paths are unavailable and is this supposed so?

# 1.

In [ ]:
# for i in df_ivs.index:
#     org = df_ivs.origin[i]
#     org = df_h.loc[df_h.harbour_code == org]['harbour_node'].values[0]
#     dest = df_ivs.destination[i]
#     dest = df_h.loc[df_h.harbour_code == dest]['harbour_node'].values[0]
#     try:
#         nx.dijkstra_path(G, org, dest, weight='length_m')
#     except:
#         print("Path between", dest, "and", org, "not feasible")

Looks okay, all path of df_ivs are available

In [ ]:
# types = list(df_links.Code.unique())
# types.remove('_0')
# types = sorted(types[1:])
# print(types)
# type_dict={}
# #minus 1 because last one is open class anyway
# for i in range(len(types)-1):
#     type_dict[i] = types[:(i+1)]
# type_dict[8]=types

In [ ]:
# type_dict

In [ ]:
# # code below was used to conclude that it was necessary to consider different routes for different ships.
# # does not work after network cleaning though, but principle of step 3 is the same

In [ ]:
# #To what extent to paths differ for these OD pairs for different boat types?
# for i in df_ivs.index:
#
#     org_n = df_ivs.origin[i]
#     org = df_h.loc[df_h.harbour_code == org_n]['harbour_node'].values[0]
#     dest_n = df_ivs.destination[i]
#     dest = df_h.loc[df_h.harbour_code == dest_n]['harbour_node'].values[0]
#
#     org_route = nx.dijkstra_path(G, org, dest, weight='length_m')
#     for type_index, types_exc in type_dict.items():
#
#         #copy graph
#         H = G.copy()
#
#         #determine links that are available based on dict (e.g. not of the type in type_exc list)
#         unavailable_edges = df_links.loc[df_links.Code.isin(types_exc)]
#         #now remove those edges
#         for link in unavailable_edges.index:
#             H.remove_edge(unavailable_edges.source[link], unavailable_edges.target[link])
#
#         if nx.has_path(H, org, dest):
#             route_type = nx.dijkstra_path(H, org, dest, weight='length_m')
#             if route_type != org_route:
#                 print("Ships of type", set(types)-set(types_exc), "must take other route for path", org_n, dest_n)
#                 org_route = route_type
#         else:
#             print("Path between", org_n, "and", dest_n, "not feasible for types", set(types)-set(types_exc))
#             break

Based on these observations, step 3: flow splitting was inserted above